In [208]:
import numpy as np
import pandas as pd
import random

In [20]:
#Implementamos todas las funciones a utilizar
def sphere(x):
    x = np.array(x)
    return sum(x**2)

def ackley(x):
    x = np.array(x)
    t1 = -0.2*np.sqrt(np.mean(x**2))
    t2 = np.mean(np.cos(2*np.pi*x))
    y = 20+np.exp(1)-20*np.exp(t1)-np.exp(t2)
    return y

def griewank(x):
    x = np.array(x)
    i = np.array(range(1,len(x)+1))
    t1 = sum(x**2)/4000
    t2 = np.prod(np.cos(x/np.sqrt(i)))
    return 1 + t1 - t2

def rastrigin(x):
    x = np.array(x)
    t1 = 10*len(x)
    t2 = sum(x**2-10*np.cos(2*np.pi*x))
    return t1 + t2

#La función aplica para parametros mayores o iguales a dos dimensiones
def rosenbrock(x):
    x = np.array(x)
    x1 = x[1:]
    x0 = np.delete(x,-1)
    t1 = 100*(x1-x0**2)**2
    t2 = (1-x0)**2
    return sum(t1 + t2)

In [21]:
#Utilizamos las funciones anteriormente programadas
def codifica_aux(decimal,nBit):
    binary = np.zeros(nBit,dtype=int)
    i = nBit - 1
    while decimal > 0:
        binary[i] = decimal % 2
        decimal //=2
        i-=1
    return binary

def decodifica_aux(binary):
    decimal = 0 
    for i in range(len(binary)):
        decimal += binary[i]*2**(len(binary)-i-1)
    return decimal

def codifica(x,nBit,a,b):
    precision = (b-a)/(2**nBit)
    x= max(a,min(b,x))
    index = int((x-a)/precision)
    binary = codifica_aux(index,nBit)
    return binary

def decodifica(x_cod,nBit,a,b): 
    precision = (b-a)/(2**nBit)
    index = decodifica_aux(x_cod)
    x = a+ index *precision
    return x

def codifica_v(x,nBit,a,b):
    binary = list()
    for i in x:
        binary.append(codifica(i,nBit,a,b))
    return binary

def decodifica_v(x_cod,nBit,a,b):
    decimal = list()
    for i in x_cod:
        decimal.append(decodifica(i,nBit,a,b))
    return decimal 

In [290]:
def busqueda_aleatoria(funcion_objetivo, dimension, iteraciones, intervalo):
    mejor_solucion = np.random.uniform(intervalo[0], intervalo[1], dimension)
    mejor_evaluacion = funcion_objetivo(mejor_solucion)
    
    for i in range(iteraciones):
        solucion = np.random.uniform(intervalo[0], intervalo[1], dimension)
        evaluacion = funcion_objetivo(solucion)
        if evaluacion < mejor_evaluacion:
            mejor_solucion = solucion
            mejor_evaluacion = evaluacion
            
    return mejor_solucion, mejor_evaluacion

In [23]:
def vecino(solucion):
    neighbor = np.copy(solucion)
    #Ya que tenemos soluciones binarias y queremos generar un vecino de manera aleatoria
    #basta con generar el cambio de un bit de forma aleatoria
    indice = np.random.randint(len(neighbor))
    neighbor[indice] = 1 - neighbor[indice]
    return neighbor

In [97]:
#Utilizamos un esquema de enfriamiento con decremento lento para
#para poder explorar diferentes soluciones
def esquema_enfriamiento(temp,beta=0.01):
    return temp/(1+beta*temp)

In [293]:
def recocido_simulado(temperatura_inicial,temperatura_final,funcion_objetivo, dimension, intervalo,beta=0.01):
    mejor_solucion = np.random.uniform(intervalo[0], intervalo[1], dimension)
    mejor_evaluacion = funcion_objetivo(mejor_solucion)
    #Codificamos la solucion para poder generar un vecino y realizar la pertubacion
    solucion = codifica_v(mejor_solucion,100,intervalo[0],intervalo[1])
    temperatura = temperatura_inicial

    while temperatura > temperatura_final:
        #Generamos el vecino a evaluar
        nueva_solucion_cod = vecino(solucion)
        #Decodificamos para evaluar la solucion 
        nueva_solucion = decodifica_v(nueva_solucion_cod,100,intervalo[0],intervalo[1])
        nueva_evaluacion = funcion_objetivo(nueva_solucion)
        delta = nueva_evaluacion - mejor_evaluacion
        probabilidad = np.exp(-delta/temperatura)
        #Debido a que no todas las funciones necesitan perturbaciones similares
        #establecemos un criterio de aceptacion de probabilidad arbitrario aleatorio
        criterio = random.random()

        if nueva_evaluacion < mejor_evaluacion:
            solucion = codifica_v(nueva_solucion,100,intervalo[0],intervalo[1])
            mejor_evaluacion = nueva_evaluacion
            #Si la probabilidad de que la solucion sea mejor es mayor a nuestro criterio
            #aleatorio, entonces aceptamos la nueva solucion generada
        elif probabilidad > criterio:
            solucion = codifica_v(nueva_solucion,100,intervalo[0],intervalo[1])
            mejor_evaluacion = nueva_evaluacion

        #Aplicamos el esquema de enfriamiento para reducir la temperatura constantemente
        temperatura = esquema_enfriamiento(temperatura,beta)

    mejor_solucion = decodifica_v(solucion,100,intervalo[0],intervalo[1])
    
    return mejor_solucion, mejor_evaluacion

In [308]:
def resultados(funcion_objetivo,dimension,iteraciones,intervalo):
    soluciones_aleatorias = []
    soluciones_rs = []
    resultados_aleatorios = []
    resultados_rs = []
    for i in range(iteraciones):
        #Decidimos utilizar 100 iteraciones para la busqueda local iterada para balancear el uso de
        #recursos en ambos metodos
        sol_aleatoria = busqueda_aleatoria(funcion_objetivo, dimension, 10, intervalo)
        #Utilizamos los parametros de temperatura inicial y temperatura final como constantes
        sol_rs = recocido_simulado(100,0.1,funcion_objetivo,10, intervalo,beta=0.01)
        soluciones_aleatorias.append(sol_aleatoria[0])
        soluciones_rs.append(sol_rs[0])
        resultados_aleatorios.append(sol_aleatoria[1])
        resultados_rs.append(sol_rs[1])

    return soluciones_aleatorias, soluciones_rs, resultados_aleatorios, resultados_rs

In [304]:
#Elegimos un intervalo que aplique para todas
intervalo = [-5.12,5.12]
dimension = 10
iteraciones = 10

In [310]:
resultadosSphere = resultados(sphere, dimension, iteraciones, intervalo)

In [312]:
resultadosAckley = resultados(ackley, dimension, iteraciones, intervalo)

In [313]:
resultadosGriewank = resultados(griewank, dimension, iteraciones, intervalo)

In [314]:
resultadosRastrigin = resultados(rastrigin, dimension, iteraciones, intervalo)

In [315]:
resultadosRosenbrock = resultados(rosenbrock, dimension, iteraciones, intervalo)

C:\Users\danaa\AppData\Local\Temp\ipykernel_23648\3208883603.py:12: RuntimeWarning: overflow encountered in exp
  probabilidad = np.exp(-delta/temperatura)


In [338]:
#Funcion auxiliar para generar los .txt de las soluciones
def solucion_text(funcion,solutions):
    i = 1
    for solution in solutions:
        nombre_archivo = "sol_"+funcion+"_"+str(i)+".txt"
        i += 1
        with open(nombre_archivo, "w") as archivo:
            for sol in solution:
                archivo.write(str(sol) + "\n")

In [339]:
#Guardamos las soluciones generadas
solucion_text("Sphere",resultadosSphere[1])
solucion_text("Ackley",resultadosAckley[1])
solucion_text("Griewank",resultadosGriewank[1])
solucion_text("Rastrigin",resultadosRastrigin[1])
solucion_text("Rosenbrock",resultadosRosenbrock[1])

In [340]:
#Calculamos las estadisticas
def estadisticas(solutions):
    statistics = [min(solutions),np.mean(solutions),max(solutions)]
    return statistics

In [351]:
#Calculamos para cada una de las funciones
Sphere_aleatorio = estadisticas(resultadosSphere[2])
Sphere_rs = estadisticas(resultadosSphere[3])
Ackley_aleatorio = estadisticas(resultadosAckley[2])
Ackley_rs = estadisticas(resultadosAckley[3])
Griewank_aleatorio = estadisticas(resultadosGriewank[2])
Griewank_rs = estadisticas(resultadosGriewank[3])
Rastrigin_aleatorio = estadisticas(resultadosRastrigin[2])
Rastrigin_rs = estadisticas(resultadosRastrigin[3])
Rosenbrock_aleatorio = estadisticas(resultadosRosenbrock[2])
Rosenbrock_rs = estadisticas(resultadosRosenbrock[3])

In [352]:
print("Sphere")
print("Busqueda aleatoria " + str(Sphere_aleatorio))
print("Recocido simulado " + str(Sphere_rs))
print("Ackley")
print("Busqueda aleatoria " + str(Ackley_aleatorio))
print("Recocido simulado " + str(Ackley_rs))
print("Griewank")
print("Busqueda aleatoria " + str(Griewank_aleatorio))
print("Recocido simulado " + str(Griewank_rs))
print("Rastrigin")
print("Busqueda aleatoria " + str(Rastrigin_aleatorio))
print("Recocido simulado " + str(Rastrigin_rs))
print("Rosenbrock")
print("Busqueda aleatoria " + str(Rosenbrock_aleatorio))
print("Recocido simulado " + str(Rosenbrock_rs))

Sphere
Busqueda aleatoria [26.757718712499337, 49.24184979081719, 65.88856262087144]
Recocido simulado [35.6454713697669, 95.16345770634614, 136.29266921532593]
Ackley
Busqueda aleatoria [7.823459672279524, 8.934621475647587, 9.533788664288885]
Recocido simulado [7.717322946856633, 10.297152829894232, 11.939538319525514]
Griewank
Busqueda aleatoria [0.9990651242946653, 1.0076333571745864, 1.0160906623617254]
Recocido simulado [1.0119226100822711, 1.0187104069256598, 1.0293249775870337]
Rastrigin
Busqueda aleatoria [112.66269726883138, 137.77819455647835, 183.18828556651647]
Recocido simulado [133.54014588713338, 176.35250396516128, 251.83324545629065]
Rosenbrock
Busqueda aleatoria [12374.013000662266, 45283.92242801597, 86374.48415491161]
Recocido simulado [16312.507539063667, 84545.11550079465, 210350.81593012394]
